In [ ]:
# montowanie google dysku #
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
COLAB = True
print("Note: using Google CoLab")

# formatowanie czasu #
import time 
def time_elapsed(sec):
  h = int(sec/3600)
  m = int(sec/60)
  s = sec % 60
  return "{}:{>02}:{:>05.2f}".format(h,m,s)

Mounted at /content/drive
Note: using Google CoLab


In [ ]:
# pobranie i formatowanie danych #
import numpy as np
import os
import pandas as pd
PATH = '/content/drive/My Drive/projects/glasses'
DATA_PATH = '/content/drive/My Drive/projects/glasses/train.csv'
SUBMIT_DATA_PATH = '/content/drive/My Drive/projects/glasses/test.csv'

# Read the data set
df = pd.read_csv(DATA_PATH,na_values=['NA','?'])
df_submit = pd.read_csv(SUBMIT_DATA_PATH,na_values=['NA','?'])

x_columns = df.columns.drop('id').drop('glasses')
x_submit_columns = df_submit.columns.drop('id')

x_submit = df_submit[x_submit_columns].values
x = df[x_columns].values
y = df['glasses'].values


In [ ]:
# stworzenie własnej sieci neuronowej
import tensorflow.keras.initializers
import statistics
import tensorflow.keras
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, InputLayer
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedShuffleSplit
from tensorflow.keras.layers import LeakyReLU,PReLU
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import math

def mlogloss(y_test, preds): # funckja 
    epsilon = 1e-15
    sum = 0
    for row in zip(preds,y_test):
        x = row[0][row[1]]
        x = max(epsilon,x)
        x = min(1-epsilon,x)
        sum+=math.log(x)
    return( (-1/len(preds))*sum)

def build_snn(input_size):
    neuronPct = 0.559  # parametry zooptymalizowane za pomoca BayesianOplimization
    neuronShrink = 0.3227
    dropout = 0.1526
    lr = 0.06767
    neuronCount = int(neuronPct * 1000)   
    model = Sequential()
    layer = 0
    while neuronCount>25 and layer<10:
        # The first (0th) layer needs an input input_dim(neuronCount)
        if layer==0:
            model.add(Dense(neuronCount,input_dim=input_size,activation=PReLU()))
        else:
            model.add(Dense(neuronCount, activation=PReLU())) 
        layer += 1
        # Add dropout after each hidden layer
        model.add(Dropout(dropout))
        # Shrink neuron count for each layer
        neuronCount = neuronCount * neuronShrink
    model.add(Dense(1,activation='sigmoid')) # Output
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=lr))
    return model

def blend_ensamble(x,y,x_submit):
  FOLDS = 10 # podzielenie danych treningowych na 10 czesci
  kf = StratifiedKFold(FOLDS)
  folds = list(kf.split(x,y))

  models = [
            KerasClassifier(build_fn = build_snn, input_size=x.shape[1]),
            RandomForestClassifier(n_estimators=100,n_jobs=-1,criterion='gini'),
            RandomForestClassifier(n_estimators=100,n_jobs=-1,criterion='entropy'),
            ExtraTreesClassifier(n_estimators=100,n_jobs=-1,criterion='gini'),
            ExtraTreesClassifier(n_estimators=100,n_jobs=-1,criterion='entropy'),
            GradientBoostingClassifier(learning_rate=0.05,subsample=0.5,max_depth=6,n_estimators=50)]

  dataset_blend_train = np.zeros((x.shape[0], len(models))) # każdy model wypracuje jedna wartosc dla jednej danej
  dataset_blend_test = np.zeros((x_submit.shape[0], len(models))) 

  for j, model in enumerate(models): # dla każdego modelu
    print(f'{j} Model: {model}')
    fold_sums = np.zeros((x_submit.shape[0],len(folds))) # dla każdego zestawu danych wyznaczone beda predyckje i wysrednione
    total_loss = 0
    for i, (train,test) in enumerate(folds):  # dla każdego zestawu danych treningowych
      x_train = x[train]
      y_train = y[train]
      x_test = x[test] # x testowe ale ze zbioru treningowego
      y_test = y[test]
      if j == 0:
        model.fit(x_train,y_train,epochs=50,verbose=0)
      else:
        model.fit(x_train,y_train)
      pred = np.array(model.predict_proba(x_test))
      pred_submit = np.array(model.predict_proba(x_submit))
      dataset_blend_train[test,j] = pred[:,1] # zapisanie predykcji dla danego modelu oraz danego zestawu danych
      fold_sums[:,i] = pred_submit[:,1] # dla kazdego zestawu danych policzyc predykcje
      loss = mlogloss(y_test,pred) # policzenie pomocniczej funkcji celu
      total_loss+=loss # liczenie sumy funkcji celu dla modelu
      print(f'Fold {i} has loss: {loss}') 
    print("{}: Mean loss={}".format(model.__class__.__name__,total_loss/len(folds)))
    dataset_blend_test[:,j] = fold_sums.mean(1) # wyznaczenie jednej predykcji dla jednego modelu
  
  print('Blending Models')
  # wyznaczenie ostatecznej predykcji za pomoca regresji logistycznej
# ostateczne predykcja na podstawie predykcji wszystkich modeli
  return dataset_blend_train,dataset_blend_test
def stretch(y):
    return (y - y.min()) / (y.max() - y.min())

SyntaxError: ignored

In [ ]:
(dataset_blend_train,dataset_blend_test) = blend_ensamble(x,y,x_submit)
#submit_data = stretch(submit_data)

blend = LogisticRegression(solver='lbfgs') 
blend.fit(dataset_blend_train,y) 
wynik=blend.predict_proba(dataset_blend_test) 

submit_data = stretch(wynik)

    ####################
    # Build submit file
    ####################
ids = [id+1 for id in range(submit_data.shape[0])]
submit_filename = os.path.join(PATH, "glasses_submit.csv")
submit_df = pd.DataFrame({'FaceID': ids, 
                              'GlassesProbability': 
                              submit_data[:, 1]},
                             columns=['FaceID',
                            'GlassesProbability'])
submit_df.to_csv(submit_filename, index=False)

0 Model: <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f5ddd4efbe0>
Fold 0 has loss: 0.3070113457325397
Fold 1 has loss: 0.4605170185988092
Fold 2 has loss: 1.3815510557964268
Fold 3 has loss: 0.07675283643313487
Fold 4 has loss: 0.6907755278982138
Fold 5 has loss: 0.1535056728662701
Fold 6 has loss: 0.537269855031944
Fold 7 has loss: 0.0767528364331349
Fold 8 has loss: 0.4605170185988091
Fold 9 has loss: 0.23025850929940458
KerasClassifier: Mean loss=0.43749116766886875
1 Model: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_stat

NameError: ignored

In [ ]:

#submit_data= np.around([submit_data])

PATH = '/content/drive/My Drive/projects/glasses'

ids = [id+4501 for id in range(submit_data.shape[0])]
submit_filename = os.path.join(PATH, "glasses_submit.csv")
submit_df = pd.DataFrame({'ID': ids, 
                              'Glasses': 
                              submit_data[:, 1]},
                             columns=['ID',
                            'Glasses'])
submit_df.to_csv(submit_filename, index=False)

In [ ]:

submit_data.shape

(500, 2)